In [2]:
# Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched 
from sahi.prediction import visualize_object_predictions
from IPython.display import Image
from numpy import asarray
import cv2
import os

In [3]:
# Download YOLOv8-S model to 'models/yolov8s.pt'
yolov11_model_path = 'models/yolo11/last.pt'
yolov8_model_path = 'models/yolov8/last.pt'
#download_yolov8s_model(destination_path=yolov8_model_path)

In [4]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [5]:
slice_detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [15]:
import cv2
from PIL import Image

def get_slice_parameters(object_density):
    
    #image_path = "test_data/0000006_06773_d_0000018.jpg"
    #image = Image.open(image_path).convert("RGB")
    #image_width, image_height  = image.size
    #slice_width = image_width
    #slice_height = image_height
    #overlap_width_ratio = 0.0
    #overlap_height_ratio = 0.0

    if object_density >= 50:
        slice_width = 256
        slice_height = 256
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        slice_width = 256
        slice_height = 256
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        slice_width = 256
        slice_height = 256
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        slice_width = 1024
        slice_height = 1024
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [7]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [8]:
# Function to get image details by image_id
def get_image_info(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image
    return None

In [9]:
import json
dataset_json_path = "./subset_visdrone_test_data.json"
with open(dataset_json_path, "r") as file:
    data = json.load(file)

In [17]:
image_path = 'test_data/0000006_05208_d_0000014.jpg'
#image_as_pil = read_image_as_pil(image_path)
filename_without_ext = '0000006_05208_d_0000014'#Path(filename).stem

print("*****************************************")
print("File Name", filename_without_ext)

img = get_image_info(data, filename_without_ext)

img['height'], img['id'], img

*****************************************
File Name 0000006_05208_d_0000014


(765,
 324,
 {'height': 765,
  'width': 1360,
  'id': 324,
  'file_name': 'images/0000006_05208_d_0000014.jpg'})

In [29]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult

# export visualization
def predict_sliced_images_modified(input_folder, dataset_json_path, detection_model, slice_detection_model):
    """
    Processes all image files in input_folder:
      - Runs predictions using get_prediction function and detection_model.
      - Saves annotated images with bounding boxes in output_folder.
      - Saves prediction details as JSON files in output_folder.
    
    Parameters:
      input_folder (str): Path to the folder containing images.
      output_folder (str): Path to the folder where results will be saved.
      detection_model: Your detection model used for prediction.
    """
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
    
    #color = (0, 255, 0)  # original annotations in green
    visual_bbox_gt_thickness = 3
    visual_bbox_thickness = 2
    visual_text_size = 0.5
    visual_text_thickness = 1
    visual_hide_labels = False
    visual_hide_conf = False
    visual_export_format = 'png'
    sliced_predictions = []
    image_ids = []
    coco_json = []
    
    # Loop over files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, filename)
            image_as_pil = read_image_as_pil(image_path)
            filename_without_ext = Path(filename).stem
            
            print("*****************************************")
            print("File Name", filename_without_ext)

            img_id = get_image_id(data, filename_without_ext)
            #image_ids.append(image_id)
            
            # Get predictions from your detection model
            time_start = time.time()
            prediction = get_prediction(image_path, detection_model)
            time_end = time.time() - time_start
            #print(f"Prediction Performed in {time_end} seconds")
            print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))
            
            object_density = len(prediction.object_prediction_list)
            print("Object Density:", object_density)
            
            slice_width, slice_height, overlap_width_ratio, overlap_height_ratio = get_slice_parameters(object_density)

            print("********* Slice Parameters ***********")
            print("Slice Width: ", slice_width)
            print("Slice Height: ", slice_height)
            print("Overlap Width Ratio: ", overlap_width_ratio)
            print("Overlap Height Ratio: ", overlap_height_ratio)

            time_start_slice = time.time()
            result_sahi = get_sliced_prediction(
                image_path,
                slice_detection_model,
                slice_height = slice_height,
                slice_width = slice_width,
                overlap_height_ratio = overlap_height_ratio,
                overlap_width_ratio = overlap_width_ratio,
                postprocess_type = "OptNMS",
                verbose = 2
            )
            time_end_slice = time.time() - time_start_slice
            #print(f"Prediction Performed in {time_end1} seconds")
            print("Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
            
            coco_prediction = result_sahi.to_coco_predictions(image_id=img_id)

            for idx, predict in enumerate(coco_prediction):
                if coco_prediction[idx]["bbox"]:
                        coco_json.append(predict)
                
            sliced_predictions.append(result_sahi)
            
            visualize_object_predictions(
                np.ascontiguousarray(image_as_pil),
                object_prediction_list=result_sahi.object_prediction_list,
                rect_th=visual_bbox_thickness,
                text_size=visual_text_size,
                text_th=visual_text_thickness,
                hide_labels=visual_hide_labels,
                hide_conf=visual_hide_conf,
                output_dir=save_dir,
                file_name=filename_without_ext,
                export_format=visual_export_format,
            )
            
        total_time = time_end + time_end_slice
     
    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(coco_json, save_path)
        print(f"Prediction results are successfully exported to {save_dir}")
    print(f"Prediction Completed Sucessfully: {len(sliced_predictions)} images")
    print("Total Prediction time is: {:.2f} ms".format(total_time * 1000))
    return sliced_predictions

In [27]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult

# export visualization
def predict_sliced_images(input_folder, dataset_json_path, detection_model, slice_detection_model):
    """
    Processes all image files in input_folder:
      - Runs predictions using get_prediction function and detection_model.
      - Saves annotated images with bounding boxes in output_folder.
      - Saves prediction details as JSON files in output_folder.
    
    Parameters:
      input_folder (str): Path to the folder containing images.
      output_folder (str): Path to the folder where results will be saved.
      detection_model: Your detection model used for prediction.
    """
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
    
    #color = (0, 255, 0)  # original annotations in green
    visual_bbox_gt_thickness = 3
    visual_bbox_thickness = 2
    visual_text_size = 0.5
    visual_text_thickness = 1
    visual_hide_labels = False
    visual_hide_conf = False
    visual_export_format = 'png'
    sliced_predictions = []
    image_ids = []
    coco_json = []
    
    # Loop over files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, filename)
            image_as_pil = read_image_as_pil(image_path)
            filename_without_ext = Path(filename).stem
            image_width, image_height = image_as_pil.size
            print("*****************************************")
            print("File Name", filename_without_ext)
            #print("Img Ht: ", image_height)
            img_id = get_image_id(data, filename_without_ext)
            #image_ids.append(image_id)
            
            # Get predictions from your detection model
            time_start = time.time()
            prediction = get_prediction(image_path, detection_model)
            time_end = time.time() - time_start
            #print(f"Prediction Performed in {time_end} seconds")
            print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))
            
            object_density = len(prediction.object_prediction_list)
            print("Object Density:", object_density)
            
            if object_density > 5:
                slice_width, slice_height, overlap_width_ratio, overlap_height_ratio = get_slice_parameters(object_density)
    
                print("********* Slice Parameters ***********")
                print("Slice Width: ", slice_width)
                print("Slice Height: ", slice_height)
                print("Overlap Width Ratio: ", overlap_width_ratio)
                print("Overlap Height Ratio: ", overlap_height_ratio)

                time_start_slice = time.time()
                result_sahi = get_sliced_prediction(
                    image_path,
                    slice_detection_model,
                    slice_height = slice_height,
                    slice_width = slice_width,
                    overlap_height_ratio = overlap_height_ratio,
                    overlap_width_ratio = overlap_width_ratio,
                    postprocess_type = "OptNMS",
                    verbose = 2
                )
                time_end_slice = time.time() - time_start_slice
                #print(f"Prediction Performed in {time_end1} seconds")
                print("Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                
                coco_prediction = result_sahi.to_coco_predictions(image_id=img_id)

                for idx, predict in enumerate(coco_prediction):
                    if coco_prediction[idx]["bbox"]:
                            coco_json.append(predict)
                    
                sliced_predictions.append(result_sahi)
                
                visualize_object_predictions(
                    np.ascontiguousarray(image_as_pil),
                    object_prediction_list=result_sahi.object_prediction_list,
                    rect_th=visual_bbox_thickness,
                    text_size=visual_text_size,
                    text_th=visual_text_thickness,
                    hide_labels=visual_hide_labels,
                    hide_conf=visual_hide_conf,
                    output_dir=save_dir,
                    file_name=filename_without_ext,
                    export_format=visual_export_format,
                )
                
            else:
                print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                
                coco_prediction = prediction.to_coco_predictions(image_id=img_id)

                for idx, predict in enumerate(coco_prediction):
                    if coco_prediction[idx]["bbox"]:
                            coco_json.append(predict)
                    
                sliced_predictions.append(prediction)
                
                visualize_object_predictions(
                    np.ascontiguousarray(image_as_pil),
                    object_prediction_list=prediction.object_prediction_list,
                    rect_th=visual_bbox_thickness,
                    text_size=visual_text_size,
                    text_th=visual_text_thickness,
                    hide_labels=visual_hide_labels,
                    hide_conf=visual_hide_conf,
                    output_dir=save_dir,
                    file_name=filename_without_ext,
                    export_format=visual_export_format,
                )
        total_time = time_end + time_end_slice
     
    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(coco_json, save_path)
        print(f"Prediction results are successfully exported to {save_dir}")
    print(f"Prediction Completed Sucessfully: {len(sliced_predictions)} images")
    print("Total Prediction time is: {:.2f} ms".format(total_time * 1000))
    return sliced_predictions

In [14]:
result_predict_nms = predict(source='./single_test/test',
                         dataset_json_path = './subset_vis_test_data_889.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000074_08202_d_0000016
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images:   0%|          | 0/1 [00:01<?, ?it/s]

Original Prediction Count 561
Final Bounding Box Count (NMS):  89
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  64
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  6
Prediction time is: 1023.01 ms


Performing inference on images: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]

Prediction results are successfully exported to runs/predict/exp313
Model loaded in 0.0296480655670166 seconds.
Slicing performed in 0.002722501754760742 seconds.
Prediction performed in 1.0230062007904053 seconds.
Exporting performed in 0.08468127250671387 seconds.


In [30]:
# Example usage: Adaptive-NMS
source_folder = './single_test/test'
json_path = "./subset_vis_test_data_889.json"
result_preds_adaptive_single_model_11 = predict_sliced_images_modified(source_folder, json_path, detection_model, detection_model)

UnboundLocalError: local variable 'time_end' referenced before assignment

In [13]:
result_predict_nms = predict(source='./single_test/test',
                         dataset_json_path = './subset_vis_test_data_889.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000074_08202_d_0000016
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/1 [00:03<?, ?it/s]

Original Prediction Count 869
Final Bounding Box Count (NMS):  124
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Prediction time is: 3086.98 ms


Performing inference on images: 100%|██████████| 1/1 [00:03<00:00,  3.34s/it]

Prediction results are successfully exported to runs/predict/exp312
Model loaded in 0.03259158134460449 seconds.
Slicing performed in 0.0022890567779541016 seconds.
Prediction performed in 3.0869834423065186 seconds.
Exporting performed in 0.09447598457336426 seconds.


In [8]:
# Example usage: Adaptive-NMS
source_folder = './single_test/test'
json_path = "./subset_vis_test_data_889.json"
result_preds_adaptive_single_model_11 = predict_sliced_images(source_folder, json_path, detection_model, detection_model)

*****************************************
File Name 0000074_08202_d_0000016
Initial Prediction time is: 36.44 ms
Object Density: 50
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
POST PROCESS:  OptNMS
Performing prediction on 28 slices.
Original Prediction Count 672
Filtered Prediction:  658
Final Bounding Box Count: 126
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  10
Final Bounding Box Count: 6
Filtered Prediction:  3
Final Bounding Box Count: 1
Slicing performed in 0.010401248931884766 seconds.
Prediction performed in 0.5085651874542236 seconds.
Prediction time is: 516.98 ms
Prediction results are successfully exported to sliced_predictions/exp123
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 553.42 ms


In [130]:
# Example usage: Modified Adaptive-NMS
source_folder = './test_data/images'
json_path = "./subset_visdrone_test_data.json"
result_preds_adaptive_single_model_different_conf3 = predict_sliced_images_modified(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 9999938_00000_d_0000207
Initial Prediction time is: 91.17 ms
Object Density: 24
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [888, 0]
CLASS TYPE:  False
Original Prediction Count 103
****non_class_agnostic****
Filtered Prediction:  41
Final Bounding Box Count: 40
Filtered Prediction:  43
Final Bounding Box Count: 41
Filtered Prediction:  7
Final Bounding Box Count: 6
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  9
Final Bounding Box Count: 9
Filtered Prediction:  2
Final Bounding Box Count: 2
Slicing performed in 0.008027076721191406 seconds.
Prediction performed in 0.1379075050354004 seconds.
Prediction time is: 144.71 ms
*****************************************
File Name 0000006_05208_d_0000014
Initial Prediction time is: 23.82

In [131]:
#USING Adaptive modified single METHOD (201 objects)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data.json' --result_json_path './sliced_predictions/exp94/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.168
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.253
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.281
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.463
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.318
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_10.json"
result_preds_adaptive_single_model_different_conf4 = predict_sliced_images(source_folder, json_path, detection_model, detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Initial Prediction time is: 23.37 ms
Object Density: 55
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
POST PROCESS:  OptNMS
Performing prediction on 10 slices.
Original Prediction Count 246
Filtered Prediction:  33
Final Bounding Box Count: 19
Filtered Prediction:  8
Final Bounding Box Count: 1
Filtered Prediction:  11
Final Bounding Box Count: 6
Filtered Prediction:  190
Final Bounding Box Count: 50
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  3
Final Bounding Box Count: 2
Slicing performed in 0.005565643310546875 seconds.
Prediction performed in 0.19089388847351074 seconds.
Prediction time is: 195.49 ms
Prediction results are successfully exported to sliced_predictions/exp124
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 218.86 ms


In [122]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_10.json"
result_preds_adaptive_single_model_different_conf3 = predict_sliced_images(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Initial Prediction time is: 24.96 ms
Object Density: 8
********* Slice Parameters ***********
Slice Width:  1024
Slice Height:  1024
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
POST PROCESS:  OptNMS
Performing prediction on 2 slices.
CLASS TYPE:  False
Original Prediction Count 55
****non_class_agnostic****
Filtered Prediction:  6
Final Bounding Box Count: 6
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  47
Final Bounding Box Count: 32
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.00618290901184082 seconds.
Prediction performed in 0.03507518768310547 seconds.
Prediction time is: 39.95 ms
Prediction results are successfully exported to sliced_predictions/exp93
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 49.91 ms


In [124]:
#USING Adaptive conf single METHOD (44 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_10.json' --result_json_path './sliced_predictions/exp93/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
Traceback (most recent call last):
  File "/home/dsu.local/bshakya/.local/bin/sahi", line 8, in <module>
    sys.exit(app())
  File "/home/dsu.local/bshakya/.local/lib/python3.9/site-packages/sahi/cli.py", line 32, in app
    fire.Fire(sahi_app)
  File "/home/dsu.local/bshakya/.local/lib/python3.9/site-packages/fire/core.py", line 135, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/home/dsu.local/bshakya/.local/lib/python3.9/site-packages/fire/core.py", line 468, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/home/dsu.local/bshakya/.local/lib/python3.9/site-packages/fire/core.py", line 684, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/home/dsu.local/bshakya/.local/lib/python3.9/site-packages/sahi/scripts/coco_evaluation.py", line 362, in evaluate
    resul

In [115]:
#USING Adaptive conf single METHOD (44 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp89/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.230
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.415
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.154
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [94]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_model_different_conf2 = predict_sliced_images(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Initial Prediction time is: 87.43 ms
Object Density: 30
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 63
****non_class_agnostic****
Filtered Prediction:  10
Final Bounding Box Count: 10
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  51
Final Bounding Box Count: 28
Slicing performed in 0.005843639373779297 seconds.
Prediction performed in 0.12724924087524414 seconds.
Prediction time is: 131.87 ms
Prediction results are successfully exported to sliced_predictions/exp85
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 219.30 ms


In [95]:
#USING Adaptive conf single METHOD (44 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp85/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.420
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.417
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [90]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_model_different_conf1 = predict_sliced_images(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Initial Prediction time is: 85.06 ms
Object Density: 30
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 101
****non_class_agnostic****
Filtered Prediction:  16
Final Bounding Box Count: 15
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  82
Final Bounding Box Count: 40
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.0058133602142333984 seconds.
Prediction performed in 0.13054871559143066 seconds.
Prediction time is: 136.18 ms
Prediction results are successfully exported to sliced_predictions/exp84
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 221.24 ms


In [91]:
#USING Adaptive conf single METHOD (44 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp84/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.259
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.751
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [84]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_model_different_conf = predict_sliced_images(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Initial Prediction time is: 23.04 ms
Object Density: 55
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
POST PROCESS:  OptNMS
Performing prediction on 10 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [768, 0]
SHIFT AMOUNT:  [256, 253]
CLASS TYPE:  False
Original Prediction Count 70
****non_class_agnostic****
Filtered Prediction:  10
Final Bounding Box Count: 10
Filtered Prediction:  3
Final Bounding Box Count: 3
Filtered Prediction:  57
Final Bounding Box Count: 31
Slicing performed in 0.005728960037231445 seconds.
Prediction performed in 0.08202123641967773 seconds.
Prediction time is: 86.65 ms
Prediction results are successfully exported to sliced_predictions/exp83
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 109.69 ms


In [87]:
#USING Adaptive conf single METHOD (44 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp83/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.417
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.091
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [73]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_model = predict_sliced_images(source_folder, json_path, slice_detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Object Density: 55
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
POST PROCESS:  OptNMS
Performing prediction on 10 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [768, 0]
SHIFT AMOUNT:  [256, 253]
CLASS TYPE:  False
Original Prediction Count 116
****non_class_agnostic****
Filtered Prediction:  22
Final Bounding Box Count: 19
Filtered Prediction:  2
Final Bounding Box Count: 1
Filtered Prediction:  3
Final Bounding Box Count: 3
Filtered Prediction:  88
Final Bounding Box Count: 40
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.006308555603027344 seconds.
Prediction performed in 0.08841061592102051 seconds.
Prediction time is: 93.34 ms
Prediction results are successfully exported to sliced_predictions/exp79
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 121.95 ms


In [75]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_model_11 = predict_sliced_images(source_folder, json_path, detection_model, detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Object Density: 44
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 83
****non_class_agnostic****
Filtered Prediction:  14
Final Bounding Box Count: 13
Filtered Prediction:  4
Final Bounding Box Count: 1
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  63
Final Bounding Box Count: 34
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.006582975387573242 seconds.
Prediction performed in 0.07649707794189453 seconds.
Prediction time is: 81.38 ms
Prediction results are successfully exported to sliced_predictions/exp80
Prediction Completed Sucessfully: 1 images
Total Prediction time is: 112.67 ms


In [76]:
#USING Adaptive single METHOD (50 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp80/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.237
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.415
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.194
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [52]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_model = predict_sliced_images(source_folder, json_path, detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Object Density: 55
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
POST PROCESS:  OptNMS
Performing prediction on 10 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [768, 0]
SHIFT AMOUNT:  [256, 253]
CLASS TYPE:  False
Original Prediction Count 116
****non_class_agnostic****
Filtered Prediction:  22
Final Bounding Box Count: 19
Filtered Prediction:  2
Final Bounding Box Count: 1
Filtered Prediction:  3
Final Bounding Box Count: 3
Filtered Prediction:  88
Final Bounding Box Count: 40
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.005620002746582031 seconds.
Prediction performed in 0.08562970161437988 seconds.
Prediction time is: 90.17 ms
Prediction results are successfully exported to sliced_predictions/exp76
Prediction Completed Sucessfully: 1 images


In [74]:
#USING Adaptive single METHOD (64 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp79/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.265
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.751
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [53]:
#USING Adaptive single METHOD (64 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp76/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.265
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.751
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [59]:
# Example usage: Adaptive-NMS (65 objects)
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_multi_model = predict_sliced_images(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Object Density: 55
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
POST PROCESS:  OptNMS
Performing prediction on 10 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [768, 0]
SHIFT AMOUNT:  [256, 253]
CLASS TYPE:  False
Original Prediction Count 111
****non_class_agnostic****
Filtered Prediction:  24
Final Bounding Box Count: 20
Filtered Prediction:  3
Final Bounding Box Count: 1
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  81
Final Bounding Box Count: 41
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.00595402717590332 seconds.
Prediction performed in 0.16543340682983398 seconds.
Prediction time is: 171.73 ms
Prediction results are successfully exported to sliced_predictions/exp77
Prediction Completed Sucessfully: 1 images


In [60]:
#USING Adaptive single METHOD (65 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp77/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.253
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.419
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.889
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [64]:
# Example usage: Adaptive-NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1563.json"
result_preds_adaptive_single_multi_first_11_model = predict_sliced_images(source_folder, json_path, detection_model, slice_detection_model)

*****************************************
File Name 0000078_03171_d_0000009
Object Density: 44
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [848, 0]
CLASS TYPE:  False
Original Prediction Count 101
****non_class_agnostic****
Filtered Prediction:  16
Final Bounding Box Count: 15
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  1
Final Bounding Box Count: 1
Filtered Prediction:  82
Final Bounding Box Count: 40
Filtered Prediction:  1
Final Bounding Box Count: 1
Slicing performed in 0.005623340606689453 seconds.
Prediction performed in 0.12740802764892578 seconds.
Prediction time is: 131.90 ms
Prediction results are successfully exported to sliced_predictions/exp78
Prediction Completed Sucessfully: 1 images


In [65]:
#USING Adaptive single METHOD (58 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './sliced_predictions/exp78/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.259
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.751
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [67]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1563.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000078_03171_d_0000009
Image Size:  (1360, 765)
POST PROCESS:  NMS
Performing prediction on 10 slices.
SHIFT AMOUNT:  [0, 0]
SHIFT AMOUNT:  [768, 0]
SHIFT AMOUNT:  [256, 253]
Total Valid prediction:  19
Total Valid prediction:  1
Total Valid prediction:  3
Total Valid prediction:  42
Total Valid prediction:  1
Total Valid prediction:  1
Prediction time is: 132.20 ms


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  4.09it/s]

Prediction results are successfully exported to runs/predict/exp168
Model loaded in 0.03319692611694336 seconds.
Slicing performed in 0.0012388229370117188 seconds.
Prediction performed in 0.13219690322875977 seconds.
Exporting performed in 0.04612278938293457 seconds.


In [68]:
#USING NMS METHOD (67 objects)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1563.json' --result_json_path './runs/predict/exp168/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.751
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50:0.95 | a